In [11]:
import sys
import zmq
import json
import pandas as pd
from pandas_gbq import read_gbq
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio

pio.renderers.default = 'notebook'  # or 'jupyterlab' if using JupyterLab


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

## Libraries

In [13]:
NODE_DOMAIN_MAP = {
	"GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTBUXVRJSXHYQ": "Stellar Development Foundation",
	"GCGB2S2KGYARPVIA37HYZXVRM2YZUEXA6S33ZU5BUDC6THSB62LZSTYH": "Stellar Development Foundation",
	"GCM6QMP3DLRPTAZW2UZPCPX2LF3SXWXKPMP3GKFZBDSF3QZGV2G5QSTK": "Stellar Development Foundation",
	"GAK6Z5UVGUVSEK6PEOCAYJISTT5EJBB34PN3NOLEQG2SUKXRVV2F6HZY": "SatoshiPay",
	"GBJQUIXUO4XSNPAUT6ODLZUJRV2NPXYASKUBY4G5MYP3M47PCVI55MNT": "SatoshiPay",
	"GC5SXLNAM3C4NMGK2PXK4R34B5GNZ47FYQ24ZIBFDFOCU6D4KBN4POAE": "SatoshiPay",
	"GCFONE23AB7Y6C5YZOMKUKGETPIAJA4QOYLS5VNS4JHBGKRZCPYHDLW7": "LOBSTR",
	"GCB2VSADESRV2DDTIVTFLBDI562K6KE3KMKILBHUHUWFXCUBHGQDI7VL": "LOBSTR",
	"GD5QWEVV4GZZTQP46BRXV5CUMMMLP4JTGFD7FWYJJWRL54CELY6JGQ63": "LOBSTR",
	"GA7TEPCBDQKI7JQLQ34ZURRMK44DVYCIGVXQQWNSWAEQR6KB4FMCBT7J": "LOBSTR",
	"GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJDHQWBR5NNK7": "LOBSTR",
	"GAAV2GCVFLNN522ORUYFV33E76VPC22E72S75AQ6MBR5V45Z5DWVPWEU": "Blockdaemon Inc.",
	"GAVXB7SBJRYHSG6KSQHY74N7JAFRL4PFVZCNWW2ARI6ZEKNBJSMSKW7C": "Blockdaemon Inc.",
	"GAYXZ4PZ7P6QOX7EBHPIZXNWY4KCOBYWJCA4WKWRKC7XIUS3UJPT6EZ4": "Blockdaemon Inc.",
	"GBLJNN3AVZZPG2FYAYTYQKECNWTQYYUUY2KVFN2OUKZKBULXIXBZ4FCT": "Public Node",
	"GCIXVKNFPKWVMKJKVK2V4NK7D4TC6W3BUMXSIJ365QUAXWBRPPJXIR2Z": "Public Node",
	"GCVJ4Z6TI6Z2SOGENSPXDQ2U4RKH3CNQKYUHNSSPYFPNWTLGS6EBH7I2": "Public Node",
	"GA7DV63PBUUWNUFAF4GAZVXU2OZMYRATDLKTC7VTCG7AU4XUPN5VRX4A": "Franklin Templeton",
	"GARYGQ5F2IJEBCZJCBNPWNWVDOFK7IBOHLJKKSG2TMHDQKEEC6P4PE4V": "Franklin Templeton",
	"GCMSM2VFZGRPTZKPH5OABHGH4F3AVS6XTNJXDGCZ3MKCOSUBH3FL6DOB": "Franklin Templeton",
	"GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7ZZAALMLNUVN": "Whalestack LLC",
	"GADLA6BJK6VK33EM2IDQM37L5KGVCY5MSHSHVJA4SCNGNUIEOTCR6J5T": "Whalestack LLC",
	"GAZ437J46SCFPZEDLVGDMKZPLFO77XJ4QVAURSJVRZK2T5S7XUFHXI2Z": "Whalestack LLC",
}

## Full history

In [17]:
nodes = NODE_DOMAIN_MAP.keys()
nodes_string = ",".join(f"'{item}'" for item in nodes)
query = f"""
  SELECT
    hl.node_id as node_id,
    hl.closed_at AS close_at
  FROM crypto-stellar.crypto_stellar.history_ledgers AS hl
  WHERE hl.closed_at BETWEEN '2024-01-01 00:00:00 UTC' AND '2025-01-01 00:00:00 UTC'
  AND hl.node_id in ({nodes_string})
"""
full_df = read_gbq(query, project_id='crypto-stellar')
full_df['close_at'] = full_df['close_at'].dt.tz_localize(None)


Downloading: 100%|█████████████████████████████████████████████████████████████████|


In [18]:
full_df

,node_id,close_at
0,GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJD...,2024-10-18 11:45:36
1,GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJD...,2024-10-18 08:49:29
2,GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJD...,2024-10-18 08:43:57
3,GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJD...,2024-10-18 10:21:58
4,GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJD...,2024-10-18 14:53:30
...,...,...
4230864,GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7Z...,2024-02-29 23:27:11
4230865,GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7Z...,2024-02-29 23:53:19
4230866,GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7Z...,2024-02-29 22:28:59
4230867,GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7Z...,2024-02-29 22:03:14


In [88]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

full_df.sort_values('close_at', inplace=True)
df = full_df[:-5000].copy()


# Drop rows with NaN values
df.dropna(inplace=True)

# Prepare the target variable
label_encoder = LabelEncoder()
df['node_id'] = label_encoder.fit_transform(df['node_id'])
df['node_id'] += 1

for lag in range(1, 10):
    df[f'lag_{lag}'] = df['node_id'].shift(lag)

df.dropna(inplace=True)

# Use lagged features as input
X = df[[f'lag_{lag}' for lag in range(1, 10)]]
y = pd.get_dummies(df['node_id'], prefix='node_id')
y = y.astype('int')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)


['GCIXVKNFPKWVMKJKVK2V4NK7D4TC6W3BUMXSIJ365QUAXWBRPPJXIR2Z'
 'GCGB2S2KGYARPVIA37HYZXVRM2YZUEXA6S33ZU5BUDC6THSB62LZSTYH'
 'GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7ZZAALMLNUVN'
 'GAZ437J46SCFPZEDLVGDMKZPLFO77XJ4QVAURSJVRZK2T5S7XUFHXI2Z'
 'GCB2VSADESRV2DDTIVTFLBDI562K6KE3KMKILBHUHUWFXCUBHGQDI7VL'
 'GADLA6BJK6VK33EM2IDQM37L5KGVCY5MSHSHVJA4SCNGNUIEOTCR6J5T'
 'GAAV2GCVFLNN522ORUYFV33E76VPC22E72S75AQ6MBR5V45Z5DWVPWEU'
 'GA7TEPCBDQKI7JQLQ34ZURRMK44DVYCIGVXQQWNSWAEQR6KB4FMCBT7J'
 'GCFONE23AB7Y6C5YZOMKUKGETPIAJA4QOYLS5VNS4JHBGKRZCPYHDLW7'
 'GAVXB7SBJRYHSG6KSQHY74N7JAFRL4PFVZCNWW2ARI6ZEKNBJSMSKW7C'
 'GCMSM2VFZGRPTZKPH5OABHGH4F3AVS6XTNJXDGCZ3MKCOSUBH3FL6DOB'
 'GA7DV63PBUUWNUFAF4GAZVXU2OZMYRATDLKTC7VTCG7AU4XUPN5VRX4A'
 'GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTBUXVRJSXHYQ'
 'GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJDHQWBR5NNK7'
 'GBJQUIXUO4XSNPAUT6ODLZUJRV2NPXYASKUBY4G5MYP3M47PCVI55MNT'
 'GCVJ4Z6TI6Z2SOGENSPXDQ2U4RKH3CNQKYUHNSSPYFPNWTLGS6EBH7I2'
 'GARYGQ5F2IJEBCZJCBNPWNWVDOFK7IBOHLJKKS

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 25s 287us/step - accuracy: 0.0480 - loss: 3.1205 - val_accuracy: 0.0484 - val_loss: 3.1200
Epoch 2/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 24s 286us/step - accuracy: 0.0481 - loss: 3.1197 - val_accuracy: 0.0474 - val_loss: 3.1199
Epoch 3/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 25s 301us/step - accuracy: 0.0480 - loss: 3.1201 - val_accuracy: 0.0483 - val_loss: 3.1199
Epoch 4/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 24s 287us/step - accuracy: 0.0478 - loss: 3.1197 - val_accuracy: 0.0481 - val_loss: 3.1200
Epoch 5/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 24s 288us/step - accuracy: 0.0478 - loss: 3.1198 - val_accuracy: 0.0483 - val_loss: 3.1198
Epoch 6/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 24s 285us/step - accuracy: 0.0484 - loss: 3.1198 - val_accuracy: 0.0476 - val_loss: 3.1198
Epoch 7/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 24s 287us/step - accuracy: 0.0481 - loss: 3.1198 - val_accuracy: 0.0476 - val_loss: 3.1198
Epoch 8/10
84518/84518 ━━━━━━━━━━━━━━━━━━━━ 25s 290us/s